In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from math import sqrt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

import torch
from torch import nn

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from multiclass_performanceMetrics import *
from dataMining_functions import *
from neuralNet_functions import *
from nested_design_analysis import *

In [3]:
from imblearn.over_sampling import SMOTE

In [4]:
# load csv files containing entropy extracted from data based on discrete wavelet transform (two finest levels)
entropy_m = pd.read_csv('features/window512_neuron_entropy_rowwindow_Molino.csv', header=None)
entropy_p = pd.read_csv('features/window512_neuron_entropy_rowwindow_Pachon.csv', header=None)
entropy_s = pd.read_csv('features/window512_neuron_entropy_rowwindow_Surface.csv', header=None)

In [5]:
entropy_m

,0,1,2,3,4,5,6,7,8,9,...,869,870,871,872,873,874,875,876,877,878
0,7.893035,6.539838,8.013306,7.729771,7.895328,7.904673,7.777006,2.092313,7.206554,7.775234,...,0,0,0,0,0,0,0,0,0,0
1,7.888959,7.481470,7.961513,7.758067,7.923035,7.882443,7.908902,7.665921,7.761139,8.042748,...,0,0,0,0,0,0,0,0,0,0
2,7.323106,7.173376,7.899712,7.980307,7.830376,7.952646,8.005411,7.939777,7.904558,7.959187,...,0,0,0,0,0,0,0,0,0,0
3,7.646228,7.791536,7.905634,7.774594,7.885257,7.999103,7.782272,6.043083,7.570184,7.514908,...,0,0,0,0,0,0,0,0,0,0
4,7.874129,7.687619,7.830016,7.672146,7.996142,7.895995,7.407081,7.692267,7.947519,7.166222,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,7.991577,7.734572,7.921799,7.928110,7.919026,7.853644,3.308346,7.914074,7.015459,7.973388,...,0,0,0,0,0,0,0,0,0,0
172,7.917425,7.828190,7.899973,5.127672,7.962174,7.854050,5.380408,4.102701,4.132467,7.940133,...,0,0,0,0,0,0,0,0,0,0
173,7.893928,5.378387,7.872902,4.426261,8.080024,7.930582,3.236722,5.033111,3.782166,7.636636,...,0,0,0,0,0,0,0,0,0,0
174,7.941442,7.748443,7.906529,6.403298,7.948029,7.843256,4.618738,3.950022,6.467974,7.901544,...,0,0,0,0,0,0,0,0,0,0


In [7]:
min_col = 275  # minimum number of neurons of a fish in the data

# slicing the data set
entropy_m = entropy_m.iloc[:, :min_col]
entropy_p = entropy_p.iloc[:, :min_col]
entropy_s = entropy_s.iloc[:, :min_col]

# class lables
# molino = 0, pachon = 1, surface = 2
entropy_m['class'] = 0
entropy_p['class'] = 1
entropy_s['class'] = 2
num_class = 3

# combine all fish classes datasets
df = pd.concat([entropy_m, entropy_p], axis=0)
df = pd.concat([df, entropy_s], axis=0)




In [8]:
# get the number of nonoverlapping window obtained from the neural signal
num_window = entropy_m.shape[0] / 16  # 16 is the number of fishes in molino group in the dataset
num_window

11.0

In [9]:
# apply balanced nested design to the dataset
# to break the dependency caused by subjects(fishes) on the neural signals of a fish
nested_design_df = balanced_nested_design_sampling(df, num_window=num_window)


/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

# Classifications

In [10]:
# shuffle data set for more randomness during training
nested_design_df = nested_design_df.sample(frac=1)
nested_design_df.head()

,0,1,2,3,4,5,6,7,8,9,...,266,267,268,269,270,271,272,273,274,class
36,3.694456,8.259718,8.229622,8.274942,8.293853,6.697897,8.147788,8.312257,8.202056,8.172234,...,8.217506,8.197120,8.196414,7.192093,8.178610,8.210772,5.173192,8.284681,7.931361,0
109,7.852640,7.894746,7.882062,7.895732,7.973520,7.910047,7.981423,8.015016,7.252604,7.862128,...,3.877100,3.157913,5.889076,4.863498,2.357794,7.916133,2.938198,7.233938,7.417195,0
10,8.153777,7.668459,8.191980,8.131147,8.284786,7.104733,8.253418,8.076950,8.256345,8.166460,...,7.885930,7.171327,3.451830,8.081379,7.903967,8.081394,7.862402,7.749086,6.496769,0
235,8.323698,8.081034,8.165765,8.130453,8.049521,8.151230,3.781471,8.196539,8.064934,8.092396,...,6.392112,3.605977,3.481117,6.676687,7.267774,5.938564,7.262753,2.921586,5.905434,1
266,7.696122,7.635213,6.723503,7.689469,7.663999,7.497981,7.593904,7.622170,7.707915,7.734548,...,6.600767,7.340059,6.720117,5.635581,7.627623,4.912814,7.556861,5.845258,6.327454,2


In [11]:
# train and test data split
X, y = nested_design_df.drop(labels='class', axis=1), nested_design_df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
# Apply SMOTE to balance the training data
# sometimes random train and test data split causes minor data imbalance
# not a big issue, but used SMOTE just in case
sm = SMOTE(random_state=42, k_neighbors=12)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [15]:
# Logistic Regression
lr_mod = LogisticRegression(max_iter=700)
lr_mod.fit(X_train, y_train)
train_pred = lr_mod.predict(X_train)
test_pred = lr_mod.predict(X_test)

In [16]:
target_names = ['Molino', 'Pachon', 'Surface']
print(classification_report(y_train, train_pred, target_names=target_names))
confusion_matrix(y_train, train_pred)

              precision    recall  f1-score   support

      Molino       1.00      1.00      1.00       100
      Pachon       1.00      1.00      1.00       100
     Surface       1.00      1.00      1.00       100

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



array([[100,   0,   0],
       [  0, 100,   0],
       [  0,   0, 100]])

In [17]:
target_names = ['Molino', 'Pachon', 'Surface']
print(classification_report(y_test, test_pred, target_names=target_names))
confusion_matrix(y_test, test_pred)

              precision    recall  f1-score   support

      Molino       0.96      1.00      0.98        27
      Pachon       1.00      1.00      1.00        21
     Surface       1.00      0.96      0.98        25

    accuracy                           0.99        73
   macro avg       0.99      0.99      0.99        73
weighted avg       0.99      0.99      0.99        73



array([[27,  0,  0],
       [ 0, 21,  0],
       [ 1,  0, 24]])

In [18]:
def SMOTE_oversample(data):
    # param: data is a dataframe (normally training data) where the target feature is in the last column
    # return balanced data by applying SMOTE oversampling technique
    # this function will be used in repeat_sampling_and_training function
    #             to apply data preprocessing for every sample
    X, y = data.iloc[:,:-1], data.iloc[:,-1]
    sm = SMOTE(random_state=42, k_neighbors=12)
    X, y = sm.fit_resample(X, y)
    df = pd.concat([X, y], axis=1)
    df = df.sample(frac=1)
    return df

In [19]:
def build_logisticRegression(**kwargs):
    # return an untrained logistic regression model
    return LogisticRegression(max_iter=500)

model_f = build_logisticRegression
predictors = list(df.drop(labels='class', axis=1).columns)  # names of the features
# random sampling and train a new model
# to more accurately test our model performace indepedent of particular samples
# Similar to the idea of K-fold cross validation

res = repeat_sampling_and_training(model_f, [], df,
                                  'class', predictors, num_repeat=100, doMinMaxScaling=False,
                                  num_window=num_window, data_processing_f=balanced_nested_design_sampling,
                                  is_oversample=True, oversample_f=SMOTE_oversample)

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

...

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.p

In [23]:
# print out model performance and confusion matrix (averaged over all sampling repetitions)
performance_dict, avg_cm = res
print_performance_metrics(performance_dict)
avg_cm

Mean of training_accuracy: 1.0
Standard deviation of training_accuracy: 0.0
Mean of testing_accuracy: 0.992844036697248
Standard deviation of testing_accuracy: 0.010014718568069852
Mean of AUC_score: 1.0
Standard deviation of AUC_score: 0.0
Mean of recall_class0: 0.993296359888421
Standard deviation of recall_class0: 0.016556637143487832
Mean of recall_class1: 0.9942617185869759
Standard deviation of recall_class1: 0.013446809462194212
Mean of recall_class2: 0.991032941747364
Standard deviation of recall_class2: 0.020899444350160768
Mean of precision_class0: 0.9935774432616776
Standard deviation of precision_class0: 0.01246996726726845
Mean of precision_class1: 0.9896011325932466
Standard deviation of precision_class1: 0.020471904114987463
Mean of precision_class2: 0.9955871782582547
Standard deviation of precision_class2: 0.012801002215330572
Mean of f1_class0: 0.9933249297719721
Standard deviation of f1_class0: 0.010503841314205842
Mean of f1_class1: 0.9917831328586828
Standard devia

,Predicted Class 0,Predicted Class 1,Predicted Class 2
Actual Class 0,36.47,0.17,0.08
Actual Class 1,0.12,35.55,0.08
Actual Class 2,0.12,0.21,36.20


In [24]:
# L1 penalty(Lasso) Logistic Regression
# to check if some neurons are responding more to the stimulation in vision
# if yes, then the corresponding column index of a neuron responds less to the stimulus
#               will have 0 coefficient in the weight matrix of model
from sklearn.model_selection import GridSearchCV

# 5-fold cross-validation based grid search
# to find the best hyper parameters for L1 penalty Logistic Regression
parameters = {'solver':['liblinear', 'saga'], 'C':[10, 1, 0.1, 0.01]}
model = LogisticRegression(max_iter=1000, penalty='l1')
gs = GridSearchCV(model, parameters, n_jobs=-1, cv=5)
gs.fit(X_train, y_train)
gs.best_params_

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/chihoonlee/opt/ana

{'C': 10, 'solver': 'saga'}

In [25]:
# train a model based on the best hyperparameter found in the given hyperparameter space
lr_mod = LogisticRegression(max_iter=1000, penalty='l1', C=10, solver='saga')
lr_mod.fit(X_train, y_train)
train_pred = lr_mod.predict(X_train)
test_pred = lr_mod.predict(X_test)

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [26]:
target_names = ['Molino', 'Pachon', 'Surface']
print(classification_report(y_train, train_pred, target_names=target_names))
confusion_matrix(y_train, train_pred)

              precision    recall  f1-score   support

      Molino       1.00      1.00      1.00       100
      Pachon       1.00      1.00      1.00       100
     Surface       1.00      1.00      1.00       100

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



array([[100,   0,   0],
       [  0, 100,   0],
       [  0,   0, 100]])

In [27]:
target_names = ['Molino', 'Pachon', 'Surface']
print(classification_report(y_test, test_pred, target_names=target_names))
confusion_matrix(y_test, test_pred)

              precision    recall  f1-score   support

      Molino       0.96      1.00      0.98        27
      Pachon       1.00      1.00      1.00        21
     Surface       1.00      0.96      0.98        25

    accuracy                           0.99        73
   macro avg       0.99      0.99      0.99        73
weighted avg       0.99      0.99      0.99        73



array([[27,  0,  0],
       [ 0, 21,  0],
       [ 1,  0, 24]])

In [28]:
w = lr_mod.coef_
eps = 1e-4
count = 0
for j in range(w.shape[1]):
    if all(w[:, j] < eps):
        count += 1
print(f"The number of components of the weight less than {eps} is {count} out of {w.shape[1]} number of features")

The number of components of the weight less than 0.0001 is 3 out of 275 number of features


The L1 penalty logistic regression model still classify eyeless fishes well. Unlike Hurst exponent features, very few neurons don't contribute in distinguishing fish types as we can see from the weight matrix of the model.

In [29]:
# KNN classifier without data balancing
knn_mod = KNeighborsClassifier()
knn_mod.fit(X_train, y_train)
train_pred = knn_mod.predict(X_train)
test_pred = knn_mod.predict(X_test)

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

In [30]:
print(classification_report(y_train, train_pred, target_names=target_names))
confusion_matrix(y_train, train_pred)

              precision    recall  f1-score   support

      Molino       1.00      0.98      0.99       100
      Pachon       0.93      1.00      0.97       100
     Surface       1.00      0.95      0.97       100

    accuracy                           0.98       300
   macro avg       0.98      0.98      0.98       300
weighted avg       0.98      0.98      0.98       300



array([[ 98,   2,   0],
       [  0, 100,   0],
       [  0,   5,  95]])

In [31]:
print(classification_report(y_test, test_pred, target_names=target_names))
confusion_matrix(y_test, test_pred)

              precision    recall  f1-score   support

      Molino       1.00      0.89      0.94        27
      Pachon       0.84      1.00      0.91        21
     Surface       1.00      0.96      0.98        25

    accuracy                           0.95        73
   macro avg       0.95      0.95      0.94        73
weighted avg       0.95      0.95      0.95        73



array([[24,  3,  0],
       [ 0, 21,  0],
       [ 0,  1, 24]])

In [32]:
# KNN after data balancing with multiple repetition of downsampling data for data balancing
def build_knn(**kwargs):
    # return an untrained KNN model
    return KNeighborsClassifier()

model_f = build_knn
# random sampling and train a new model
# to more accurately test our model performace indepedent of particular samples
# Similar to the idea of K-fold cross validation
res = repeat_sampling_and_training(model_f, [], df,
                                  'class', predictors, num_repeat=100, doMinMaxScaling=False,
                                  num_window=num_window, data_processing_f=balanced_nested_design_sampling,
                                  is_oversample=True, oversample_f=SMOTE_oversample)

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

...

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/chihoonlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will n

In [33]:
performance_dict, avg_cm = res
print_performance_metrics(performance_dict)
avg_cm

Mean of training_accuracy: 0.9967536189912414
Standard deviation of training_accuracy: 0.0054911977586658235
Mean of testing_accuracy: 0.9911926605504588
Standard deviation of testing_accuracy: 0.013602754490165594
Mean of AUC_score: 1.0
Standard deviation of AUC_score: 0.0
Mean of recall_class0: 0.9955497207530086
Standard deviation of recall_class0: 0.014294347104002101
Mean of recall_class1: 0.9925049825722909
Standard deviation of recall_class1: 0.016366360624898314
Mean of recall_class2: 0.98599208605364
Standard deviation of recall_class2: 0.03102696116919379
Mean of precision_class0: 0.9847960754353607
Standard deviation of precision_class0: 0.030010766205294553
Mean of precision_class1: 0.9951536507057923
Standard deviation of precision_class1: 0.01741872490820107
Mean of precision_class2: 0.9948366634499868
Standard deviation of precision_class2: 0.017267374028980806
Mean of f1_class0: 0.9898241970424366
Standard deviation of f1_class0: 0.016583971720609894
Mean of f1_class1: 

,Predicted Class 0,Predicted Class 1,Predicted Class 2
Actual Class 0,36.06,0.04,0.12
Actual Class 1,0.20,35.68,0.07
Actual Class 2,0.38,0.15,36.30


In [34]:
# SVM
svm_mod = SVC()
svm_mod.fit(X_train, y_train)
train_pred = svm_mod.predict(X_train)
test_pred = svm_mod.predict(X_test)

In [35]:
print(classification_report(y_train, train_pred, target_names=target_names))
confusion_matrix(y_train, train_pred)

              precision    recall  f1-score   support

      Molino       1.00      1.00      1.00       100
      Pachon       1.00      0.99      0.99       100
     Surface       0.99      1.00      1.00       100

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



array([[100,   0,   0],
       [  0,  99,   1],
       [  0,   0, 100]])

In [36]:
print(classification_report(y_test, test_pred, target_names=target_names))
confusion_matrix(y_test, test_pred)

              precision    recall  f1-score   support

      Molino       0.96      1.00      0.98        27
      Pachon       1.00      1.00      1.00        21
     Surface       1.00      0.96      0.98        25

    accuracy                           0.99        73
   macro avg       0.99      0.99      0.99        73
weighted avg       0.99      0.99      0.99        73



array([[27,  0,  0],
       [ 0, 21,  0],
       [ 1,  0, 24]])

In [37]:
# SVM after data balancing with multiple repetition of downsampling data for data balancing

def build_svm(**kwargs):
    # return an untrained SVM model
    return SVC(probability=True)

model_f = build_svm
# random sampling and train a new model
# to more accurately test our model performace indepedent of particular samples
# Similar to the idea of K-fold cross validation
res = repeat_sampling_and_training(model_f, [], df,
                                  'class', predictors, num_repeat=100, doMinMaxScaling=False,
                                  num_window=num_window, data_processing_f=balanced_nested_design_sampling,
                                  is_oversample=True, oversample_f=SMOTE_oversample)

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

...

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_by_class[i].drop(labels=target_name, axis=1, inplace=True)
/Users/chihoonlee/Desktop/Research/Fish_proj/Visionless_Fish/dataMining_functions.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

In [38]:
performance_dict, avg_cm = res
print_performance_metrics(performance_dict)
avg_cm

Mean of training_accuracy: 1.0
Standard deviation of training_accuracy: 0.0
Mean of testing_accuracy: 0.9978899082568807
Standard deviation of testing_accuracy: 0.007876562213534413
Mean of AUC_score: 1.0
Standard deviation of AUC_score: 0.0
Mean of recall_class0: 0.9991245421245423
Standard deviation of recall_class0: 0.005008250381597319
Mean of recall_class1: 0.9973479532163743
Standard deviation of recall_class1: 0.0123248195873656
Mean of recall_class2: 0.9972491895078102
Standard deviation of recall_class2: 0.01632212673160562
Mean of precision_class0: 0.9970758831960879
Standard deviation of precision_class0: 0.012597504530106345
Mean of precision_class1: 0.9989000189000188
Standard deviation of precision_class1: 0.006615573111316316
Mean of precision_class2: 0.9976504261327236
Standard deviation of precision_class2: 0.009318989819444503
Mean of f1_class0: 0.9980599418957629
Standard deviation of f1_class0: 0.007582599525335413
Mean of f1_class1: 0.9980847564837521
Standard devi

,Predicted Class 0,Predicted Class 1,Predicted Class 2
Actual Class 0,36.24,0.01,0.02
Actual Class 1,0.04,36.42,0.06
Actual Class 2,0.07,0.03,36.11
